This notebook serves as an example of how to load and manipulate the [Si GAP dataset](https://www.repository.cam.ac.uk/handle/1810/317974) using a `Dataset` object.

# Uncomment for Google Colab

# Imports

In [ ]:
import os
import itertools
import numpy as np

# Initialize the database

In [ ]:
from colabfit.tools.database import MongoDatabase, load_data

client = MongoDatabase('example', drop_database=True)  # drop_database=True overwrites existing

# Data loading

In [ ]:
images = load_data(
    file_path='/home/jvita/scripts/colabfit/data/gap_si/gp_iter6_sparse9k.xml.xyz',
    file_format='xyz',
    name_field=None,
    elements=['Si'],    # order matters for CFG files, but not others
    verbose=True
)
images

In [ ]:
images = list(images)

In [ ]:
images[0].info.keys()

In [ ]:
images = list(load_data(
    file_path='/home/jvita/scripts/colabfit/data/gap_si/gp_iter6_sparse9k.xml.xyz',
    file_format='xyz',
    name_field='config_type',  # key in Configuration.info to use as the Configuration name
    elements=['Si'],    # order matters for CFG files, but not others
    default_name='Si_PRX_GAP',  # default name with `name_field` not found
    verbose=True
))

In [ ]:
images[0].info['_name']

# Property definitions

In [ ]:
set(itertools.chain.from_iterable(img.arrays.keys() for img in images))

In [ ]:
set(itertools.chain.from_iterable(img.info.keys() for img in images))

In [ ]:
base_definition = {
    'property-id': 'energy-forces-stress',
    'property-title': 'Basic outputs from a static calculation',
    'property-description':
        'Energy, forces, and stresses from a calculation of a '\
        'static configuration. Energies must be specified to be '\
        'per-atom or supercell. If a reference energy has been '\
        'used, this must be specified as well.',

    'energy': {
        'type': 'float',
        'has-unit': True,
        'extent': [],
        'required': False,
        'description':
            'The potential energy of the system.'
    },
    'forces': {
        'type': 'float',
        'has-unit': True,
        'extent': [":", 3],
        'required': False,
        'description':
            'The [x,y,z] components of the force on each particle.'
    },
    'stress': {
        'type': 'float',
        'has-unit': True,
        'extent': [3, 3],
        'required': False,
        'description':
            'The full Cauchy stress tensor of the simulation cell'
    },

    'per-atom': {
        'type': 'bool',
        'has-unit': False,
        'extent': [],
        'required': True,
        'description':
            'If True, "energy" is the total energy of the system, '\
            'and has NOT been divided by the number of atoms in the '\
            'configuration.'
    },
    'reference-energy': {
        'type': 'float',
        'has-unit': True,
        'extent': [],
        'required': False,
        'description':
            'If provided, then "energy" is the energy (either of '\
            'the whole system, or per-atom) LESS the energy of '\
            'a reference configuration (E = E_0 - E_reference). '\
            'Note that "reference-energy" is just provided for '\
            'documentation, and that "energy" should already have '\
            'this value subtracted off. The reference energy must '\
            'have the same units as "energy".'
    },
}

In [ ]:
extra_stuff_definition = {
    'property-id': 'si-prx-gap-data',
    'property-title': 'Si PRX GAP data',
    'property-description': 'A property for storing all of the additional information provided for the Si PRX GAP dataset',

    'mix-history-length':         {'type': 'float',  'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'castep-file-name':           {'type': 'string', 'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'grid-scale':                 {'type': 'float',  'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'popn-calculate':             {'type': 'bool',   'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'n-neighb':                   {'type': 'int',    'has-unit': False, 'extent': [":"],   'required': False, 'description': ''},
    'oldpos':                     {'type': 'float',  'has-unit': True,  'extent': [":",3], 'required': False, 'description': ''},
    'i-step':                     {'type': 'int',    'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'md-temperature':             {'type': 'float',  'has-unit': True,  'extent': [],      'required': False, 'description': ''},
    'positions':                  {'type': 'float',  'has-unit': True,  'extent': [":",3], 'required': False, 'description': ''},
    'task':                       {'type': 'string', 'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'data-distribution':          {'type': 'string', 'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'avg-ke':                     {'type': 'float',  'has-unit': True,  'extent': [":"],   'required': False, 'description': ''},
    'force-nlpot':                {'type': 'float',  'has-unit': True,  'extent': [":",3], 'required': False, 'description': ''},
    'continuation':               {'type': 'string', 'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'castep-run-time':            {'type': 'float',  'has-unit': True,  'extent': [],      'required': False, 'description': ''},
    'calculate-stress':           {'type': 'bool',   'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'minim-hydrostatic-strain':   {'type': 'bool',   'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'avgpos':                     {'type': 'float',  'has-unit': True,  'extent': [":",3], 'required': False, 'description': ''},
    'frac-pos':                   {'type': 'float',  'has-unit': False, 'extent': [":",3], 'required': False, 'description': ''},
    'hamiltonian':                {'type': 'float',  'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'md-cell-t':                  {'type': 'float',  'has-unit': True,  'extent': [],      'required': False, 'description': ''},
    'cutoff-factor':              {'type': 'float',  'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'momenta':                    {'type': 'float',  'has-unit': False, 'extent': [":",3], 'required': False, 'description': ''},
    'elec-energy-tol':            {'type': 'float',  'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'mixing-scheme':              {'type': 'string', 'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'minim-lattice-fix':          {'type': 'float',  'has-unit': False, 'extent': [9],     'required': False, 'description': ''},
    'in-file':                    {'type': 'string', 'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'travel':                     {'type': 'float',  'has-unit': False, 'extent': [":",3], 'required': False, 'description': ''},
    'thermostat-region':          {'type': 'float',  'has-unit': False, 'extent': [":"],   'required': False, 'description': ''},
    'time':                       {'type': 'float',  'has-unit': True,  'extent': [],      'required': False, 'description': ''},
    'temperature':                {'type': 'float',  'has-unit': True,  'extent': [],      'required': False, 'description': ''},
    'kpoints-mp-grid':            {'type': 'float',  'has-unit': False, 'extent': [3],     'required': False, 'description': ''},
    'gap-force':                  {'type': 'float',  'has-unit': True,  'extent': [":",3], 'required': False, 'description': ''},
    'gap-energy':                 {'type': 'float',  'has-unit': True,  'extent': [],      'required': False, 'description': ''},
    'cutoff':                     {'type': 'float',  'has-unit': True,  'extent': [],      'required': False, 'description': ''},
    'xc-functional':              {'type': 'string', 'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'smearing-width':             {'type': 'float',  'has-unit': True,  'extent': [],      'required': False, 'description': ''},
    'pressure':                   {'type': 'float',  'has-unit': True,  'extent': [],      'required': False, 'description': ''},
    'gap-virial':                 {'type': 'float',  'has-unit': True,  'extent': [9],     'required': False, 'description': ''},
    'reuse':                      {'type': 'string', 'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'fix-occupancy':              {'type': 'bool',   'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'map-shift':                  {'type': 'float',  'has-unit': False, 'extent': [":",3], 'required': False, 'description': ''},
    'md-num-iter':                {'type': 'int',    'has-unit': False, 'extent': [], 'required': False, 'description': ''},
    'damp-mask':                  {'type': 'float',  'has-unit': False, 'extent': [":"],   'required': False, 'description': ''},
    'opt-strategy':               {'type': 'string', 'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'spin-polarized':             {'type': 'bool',   'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'nextra-bands':               {'type': 'int',    'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'fine-grid-scale':            {'type': 'float',  'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'masses':                     {'type': 'float',  'has-unit': True,  'extent': [":"],   'required': False, 'description': ''},
    'iprint':                     {'type': 'int',    'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'finite-basis-corr':          {'type': 'string', 'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'enthalpy':                   {'type': 'float',  'has-unit': True,  'extent': [],      'required': False, 'description': ''},
    'opt-strategy-bias':          {'type': 'int',    'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'force-ewald':                {'type': 'float',  'has-unit': True,  'extent': [":",3], 'required': False, 'description': ''},
    'num-dump-cycles':            {'type': 'int',    'has-unit': False,  'extent': [],     'required': False, 'description': ''},
    'velo':                       {'type': 'float',  'has-unit': True,  'extent': [":",3], 'required': False, 'description': ''},
    'md-delta-t':                 {'type': 'float',  'has-unit': True,  'extent': [],      'required': False, 'description': ''},
    'md-ion-t':                   {'type': 'float',  'has-unit': True,  'extent': [],      'required': False, 'description': ''},
    'force-locpot':               {'type': 'float',  'has-unit': True,  'extent': [":",3], 'required': False, 'description': ''},
    'numbers':                    {'type': 'int',    'has-unit': False, 'extent': [":"],   'required': False, 'description': ''},
    'max-scf-cycles':             {'type': 'int',    'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'mass':                       {'type': 'float',  'has-unit': True,  'extent': [":"],      'required': False, 'description': ''},
    'minim-constant-volume':      {'type': 'bool',   'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'cut-off-energy':             {'type': 'float',  'has-unit': True,  'extent': [],      'required': False, 'description': ''},
    'virial':                     {'type': 'float',  'has-unit': True,  'extent': [3,3],   'required': False, 'description': ''},
    'nneightol':                  {'type': 'float',  'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'max-charge-amp':             {'type': 'float',  'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'md-thermostat':              {'type': 'string', 'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'md-ensemble':                {'type': 'string', 'has-unit': False, 'extent': [],      'required': False, 'description': ''},
    'acc':                        {'type': 'float',  'has-unit': False, 'extent': [":",3], 'required': False, 'description': ''},
}

In [ ]:
client.insert_property_definition(base_definition)

In [ ]:
client.insert_property_definition(extra_stuff_definition)

# Adding to the database using `insert_data()`

In [ ]:
# Data stored on atoms needs to be cleaned
def tform(img):
    img.info['per-atom'] = False
    
    # Renaming some fields to be consistent
    info_items = list(img.info.items())
    
    for key, v in info_items:
        if key in ['_name', '_labels', '_constraints']:
            continue
            
        del img.info[key]
        img.info[key.replace('_', '-').lower()] = v

    arrays_items = list(img.arrays.items())
    for key, v in arrays_items:
        del img.arrays[key]
        img.arrays[key.replace('_', '-').lower()] = v
    
    # Converting some string values to floats
    for k in [
        'md-temperature', 'md-cell-t', 'smearing-width', 'md-delta-t',
        'md-ion-t', 'cut-off-energy', 'elec-energy-tol',
        ]:
        if k in img.info:
            try:
                img.info[k] = float(img.info[k].split(' ')[0])
            except:
                pass
    
    # Reshaping shape (9,) stress vector to (3, 3) to match definition
    if 'dft-virial' in img.info:
        img.info['dft-virial'] = img.info['dft-virial'].reshape((3,3))

In [ ]:
units = {
    'energy': 'eV',
    'forces': 'eV/Ang',
    'virial': 'GPa',
    'oldpos': 'Ang',
    'md-temperature': 'K',
    'positions': 'Ang',
    'avg-ke': 'eV',
    'force-nlpot': 'eV/Ang',
    'castep-run-time': 's',
    'avgpos': 'Ang',
    'md-cell-t': 'ps',
    'time': 's',
    'temperature': 'K',
    'gap-force': 'eV/Ang',
    'gap-energy': 'eV',
    'cutoff': 'Ang',
    'smearing-width': 'eV',
    'pressure': 'GPa',
    'gap-virial': 'GPa',
    'masses': '_amu',
    'enthalpy': 'eV',
    'force-ewald': 'eV/Ang',
    'velo': 'Ang/s',
    'md-delta-t': 'fs',
    'md-ion-t': 'ps',
    'force-locpot': 'eV/Ang',
    'mass': 'g',
    'cut-off-energy': 'eV',
    'virial': 'GPa',
}

property_map = {
    'energy-forces-stress': {
        # Property Definition field: {'field': ASE field, 'units': ASE-readable units}
        'energy': {'field': 'dft-energy', 'units': 'eV'},
        'forces': {'field': 'dft-force', 'units': 'eV/Ang'},
        'stress': {'field': 'dft-virial', 'units': 'GPa'},
        'per-atom': {'field': 'per-atom', 'units': None},
    },
    'si-prx-gap-data': {
        # k.replace('_', '-').lower(): {'field': k , 'units': units[k] if k in units else None}
        k: {'field': k , 'units': units[k] if k in units else None}
        for k in extra_stuff_definition if k not in {'property-id', 'property-title', 'property-description'}
    }
}


In [ ]:
from colabfit.tools.property_settings import PropertySettings

pso = PropertySettings(
    method='CASTEP',
    description='DFT calculations using the CASTEP software',
    files=None,
    labels=['Monkhorst-Pack'],
)

ids = client.insert_data(
    images,
    property_map=property_map,
    property_settings={'energy-forces-stress': pso, 'si-prx-gap-data': pso},
    transform=tform,
    verbose=True
)

Note: this dataset has four pairs of duplicate configurations. This can be seen by counting the number of configurations that have twice as many linked properties as expected (expected is 2).

In [ ]:
client.configurations.count_documents(
    {'relationships.properties.2': {'$exists': True}}
)

# Building Configuration Sets

In [ ]:
# Used for building groups of configurations for easier analysis/exploration
configuration_set_regexes = {
    'isolated_atom': 'Reference atom',
    'bt': 'Beta-tin',
    'dia': 'Diamond',
    'sh': 'Simple hexagonal',
    'hex_diamond': 'Hexagonal diamond',
    'bcc': 'Body-centered-cubic',
    'bc8': 'BC8',
    'fcc': 'Face-centered-cubic',
    'hcp': 'Hexagonal-close-packed',
    'st12': 'ST12',
    'liq': 'Liquid',
    'amorph': 'Amorphous',
    'surface_001': 'Diamond surface (001)',
    'surface_110': 'Diamond surface (110)',
    'surface_111': 'Diamond surface (111)',
    'surface_111_pandey': 'Pandey reconstruction of diamond (111) surface',
    'surface_111_3x3_das': 'Dimer-adatom-stacking-fault (DAS) reconstruction',
    '111adatom': 'Configurations with adatom on (111) surface',
    'crack_110_1-10': 'Small (110) crack tip',
    'crack_111_1-10': 'Small (111) crack tip',
    'decohesion': 'Decohesion of diamond-structure Si along various directions',
    'divacancy': 'Diamond divacancy configurations',
    'interstitial': 'Diamond interstitial configurations',
    'screw_disloc': 'Si screw dislocation core',
    'sp': 'sp bonded configurations',
    'sp2': 'sp2 bonded configurations',
    'vacancy': 'Diamond vacancy configurations'
}

In [ ]:
cs_ids = []

for i, (regex, desc) in enumerate(configuration_set_regexes.items()):
    co_ids = client.get_data(
        'configurations',
        fields='_id',
        query={'names': {'$regex': regex}},
        ravel=True
    ).tolist()
    
    print(f'Configuration set {i}', f'({regex}):'.rjust(22), f'{len(co_ids)}'.rjust(7))

    cs_id = client.insert_configuration_set(co_ids, description=desc)
    
    cs_ids.append(cs_id)

# Building the Dataset

In [ ]:
all_co_ids, all_pr_ids = list(zip(*ids))  # returned by insert_data()
len(all_pr_ids)

In [ ]:
ds_id = client.insert_dataset(
    cs_ids=cs_ids,
    pr_ids=all_pr_ids,
    name='Si_PRX_GAP',
    authors=[
        'Albert P. Bartók', 'James Kermode', 'Noam Bernstein', 'Gábor Csányi'
    ],
    links=[
        'https://journals.aps.org/prx/abstract/10.1103/PhysRevX.8.041048',
        'https://www.repository.cam.ac.uk/handle/1810/317974'
    ],
    description=\
        "The original DFT training data for the general-purpose silicon "\
        "interatomic potential described in the associated publication."\
        " The kinds of configuration that we include are chosen using "\
        "intuition and past experience to guide what needs to be included "\
        "to obtain good coverage pertaining to a range of properties.",
    verbose=True,
)
ds_id

# Adding labels

In [ ]:
client.apply_labels(
    dataset_id=ds_id,
    collection_name='properties',
    query={'si-prx-gap-data.xc-functional.source-value': 'PW91'},
    labels='PW91',
    verbose=True
)

client.apply_labels(
    dataset_id=ds_id,
    collection_name='properties',
    query={'si-prx-gap-data.xc-functional.source-value': 'PBE'},
    labels='PBE',
    verbose=True
)

In [ ]:
# Used to apply metadata labels to configurations for future queries
configuration_label_regexes = {
    'isolated_atom': 'isolated_atom',
    'bt': 'a5',
    'dia': 'diamond',
    'sh': 'sh',
    'hex_diamond': 'sonsdaleite',
    'bcc': 'bcc',
    'bc8': 'bc8',
    'fcc': 'fcc',
    'hcp': 'hcp',
    'st12': 'st12',
    'liq': 'liquid',
    'amorph': 'amorphous',
    'surface_001': ['surface', '001'],
    'surface_110': ['surface', '110'],
    'surface_111': ['surface', '111'],
    'surface_111_pandey': ['surface', '111'],
    'surface_111_3x3_das': ['surface', '111', 'das'],
    '111adatom': ['surface', '111', 'adatom'],
    'crack_110_1-10': ['crack', '110'],
    'crack_111_1-10': ['crac', '111'],
    'decohesion': ['diamond', 'decohesion'],
    'divacancy': ['diamond', 'vacancy', 'divacancy'],
    'interstitial': ['diamond', 'interstitial'],
    'screw_disloc': ['screw', 'dislocation'],
    'sp': 'sp',
    'sp2': 'sp2',
    'vacancy': ['diamond', 'vacancy']
}

In [ ]:
for regex, labels in configuration_label_regexes.items():
    client.apply_labels(
        dataset_id=ds_id,
        collection_name='configurations',
        query={'names': {'$regex': regex}},
        labels=labels,
        verbose=True
    )

# Next up: exploring the dataset

In [ ]:
from colabfit.tools.database import MongoDatabase, load_data

client = MongoDatabase('example')

In [ ]:
client.datasets.find_one({}, {'_id', 'name'})

In [ ]:
ds_id = '5479095385593854142'
dataset = client.get_dataset(ds_id, resync=True)['dataset']
dataset

In [ ]:
for k,v in dataset.aggregated_info.items():
    print(k, '\n\t', v)

In [ ]:
fig = client.plot_histograms(
    ['energy-forces-stress.energy', 'energy-forces-stress.forces', 'energy-forces-stress.stress'],
    yscale='log',
    ids=dataset.property_ids,
    verbose=True,
)

In [ ]:
client.dataset_to_markdown(
    ds_id=ds_id,
    base_folder='/home/jvita/scripts/colabfit/colabfit/examples/'+dataset.name,
    html_file_name='README.md',
    data_format='mongo',
    data_file_name=None,
    histogram_fields=['energy-forces-stress.energy', 'energy-forces-stress.forces', 'energy-forces-stress.stress'],
    yscale='log'
)

In [ ]:
client.get_statistics(
    ['energy-forces-stress.energy', 'energy-forces-stress.forces', 'energy-forces-stress.stress'],
    ids=dataset.property_ids,
    verbose=True
)

In [ ]:
client.configurations.find_one({'names': 'st12'}, {'_id', 'names'})

In [ ]:
conf = client.get_configuration('2813911259732539396')

from ase.visualize import view

# Creates a Jupyter Widget; may require `pip install nglview` first
view([conf], viewer='nglview')